Model Surface Output
=============================

Plot an surface map with mean sea level pressure (MSLP),
2m Temperature (F), and Wind Barbs (kt).

Imports

In [1]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndimage
from metpy.units import units
from siphon.catalog import TDSCatalog
from xarray.backends import NetCDF4DataStore
import xarray as xr

Helper functions

In [ ]:
# Helper function for finding proper time variable
def find_time_var(var, time_basename='time'):
    for coord_name in var.coordinates.split():
        if coord_name.startswith(time_basename):
            return coord_name
    raise ValueError('No time variable found for ' + var.name)

Create NCSS object to access the NetcdfSubset
---------------------------------------------
Data from NCEI GFS 0.5 deg Analysis Archive

In [ ]:
# Specify our date/time of product desired
dt = datetime(2018, 1, 4, 12)

# Construct the URL for our THREDDS Data Server Catalog,
# and access our desired dataset within via NCSS
base_url = 'https://www.ncei.noaa.gov/thredds/model-gfs-g4-anl-files-old/'
cat = TDSCatalog(f'{base_url}{dt:%Y%m}/{dt:%Y%m%d}/catalog.xml')
ncss = cat.datasets[f'gfsanl_4_{dt:%Y%m%d}_{dt:%H}00_000.grb2'].subset()

# Create our NCSS query with desired specifications
query = ncss.query()
query.time(dt)
query.lonlat_box(north=65, south=15, east=310, west=220)
query.accept('netcdf')
query.variables('Pressure_reduced_to_MSL_msl',
                'Apparent_temperature_height_above_ground',
                'u-component_of_wind_height_above_ground',
                'v-component_of_wind_height_above_ground')

# Obtain the data we've queried for
data = ncss.get_data(query)
# Make into an xarray Dataset object
ds = xr.open_dataset(NetCDF4DataStore(data)).metpy.parse_cf()

Begin data maipulation
-----------------------
Data for the surface from a model is a bit complicated.
The variables come from different levels and may have different data array shapes.

MSLP: Pressure_reduced_to_MSL_msl (time, lat, lon)

2m Temp: Apparent_temperature_height_above_ground (time, level, lat, lon)

10m Wind: u/v-component_of_wind_height_above_ground (time, level, lat, lon)

Height above ground Temp from GFS has one level (2m)

Height above ground Wind from GFS has three levels (10m, 80m, 100m)

In [ ]:
# Pull out variables you want to use
temp_level = 2 * units.meters
wind_level = 10 * units.meters

mslp = ds.Pressure_reduced_to_MSL_msl.squeeze()
temp = ds.Apparent_temperature_height_above_ground.metpy.sel(
    vertical=temp_level).squeeze()
u_wind_10m = ds['u-component_of_wind_height_above_ground'].metpy.sel(
    vertical=wind_level).squeeze()
v_wind_10m = ds['v-component_of_wind_height_above_ground'].metpy.sel(
    vertical=wind_level).squeeze()
lat = ds.lat
lon = ds.lon
time_var = ds.Pressure_reduced_to_MSL_msl.time
time = time_var.values.astype('datetime64[ms]').astype('O').squeeze()

# Combine 1D latitude and longitudes into a 2D grid of locations
lon_2d, lat_2d = np.meshgrid(lon, lat)

# Smooth MSLP a little
# Be sure to only put in a 2D lat/lon or Y/X array for smoothing
smooth_mslp = ndimage.gaussian_filter(mslp.metpy.convert_units('hPa'),
                                      sigma=3, order=0) * units.Pa

Begin map creation
------------------

In [ ]:
# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=[30, 60],
                                central_longitude=-100)

# Create new figure
fig = plt.figure(figsize=(12, 12))

# Add the map and set the extent
ax = plt.subplot(111, projection=plotcrs)
plt.title('GFS Analysis MSLP, 2m Temperature (F), Wind Barbs (kt)'
          f' {time:%d %B %Y %H:%MZ}', fontsize=16)
ax.set_extent([235., 290., 20., 55.])

# Add state boundaries to plot
ax.add_feature(cfeature.STATES.with_scale('50m'),
               edgecolor='black', linewidth=1)

# Add country borders to plot
ax.add_feature(cfeature.BORDERS.with_scale('50m'),
               edgecolor='black', linewidth=1)

# Plot MSLP Contours
clev_mslp = np.arange(0, 1200, 4)
cs = ax.contour(lon_2d, lat_2d, smooth_mslp, clev_mslp, colors='black',
                linewidths=1.5, linestyles='solid', transform=datacrs)
plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

# Plot 2m Temperature Contours
clevtemp = np.arange(-60, 101, 10)
cs2 = ax.contour(lon_2d, lat_2d, temp.metpy.convert_units(units('degF')),
                 clevtemp, colors='tab:red', linewidths=1.25,
                 linestyles='dotted', transform=datacrs)
plt.clabel(cs2, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

# Plot 10m Wind Barbs
ax.barbs(lon_2d, lat_2d,
         u_wind_10m.metpy.convert_units('kts').values,
         v_wind_10m.metpy.convert_units('kts').values,
         length=6, regrid_shape=20, pivot='middle', transform=datacrs)